In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler

from math import ceil

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.layers.advanced_activations import *
from keras.layers import Convolution1D, MaxPooling1D, AtrousConvolution1D
from keras.layers.recurrent import LSTM, GRU
from keras import regularizers

%matplotlib inline


Using TensorFlow backend.


In [2]:
train = pd.read_csv('all/sales_train.csv')
test = pd.read_csv('all/test.csv')
submission = pd.read_csv('all/sample_submission.csv')
items = pd.read_csv('all/items.csv')
item_cats = pd.read_csv('all/item_categories.csv')
shops = pd.read_csv('all/shops.csv')
print("Ok")

Ok


In [3]:
test_shops = test.shop_id.unique()
train = train[train.shop_id.isin(test_shops)]
test_items = test.item_id.unique()
train = train[train.item_id.isin(test_items)]

MAX_BLOCK_NUM = train.date_block_num.max()
MAX_ITEM = len(test_items)
MAX_CAT = len(item_cats)
MAX_YEAR = 3
MAX_MONTH = 4 # 7 8 9 10
MAX_SHOP = len(test_shops)

In [4]:
# add categories
train = train.set_index('item_id').join(items.set_index('item_id')).drop('item_name', axis=1).reset_index()


train['month'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%m'))
train['year'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y'))

In [5]:
copytrain = train

In [6]:
train = train.drop('date', axis=1)
train = train.drop('item_category_id', axis=1)
train = train.groupby(['shop_id', 'item_id', 'date_block_num', 'month', 'year']).sum()
train = train.sort_index()


In [7]:
scaler = StandardScaler()
cnt_scaler = StandardScaler()

scaler.fit(train.item_price.values.reshape(-1, 1))
cnt_scaler.fit(train.item_cnt_day.values.reshape(-1, 1))

train.item_price = scaler.transform(train.item_price.values.reshape(-1, 1))
train.item_cnt_day = cnt_scaler.transform(train.item_cnt_day.values.reshape(-1, 1))

In [8]:
price = train.reset_index().set_index(['item_id', 'shop_id', 'date_block_num'])
price = price.sort_index()

In [9]:
from sklearn import preprocessing

item_le = preprocessing.LabelEncoder()
item_le.fit(test_items)
item_dm = dict(zip(test_items, item_le.transform(test_items)))


shop_le = preprocessing.LabelEncoder()
shop_le.fit(test_shops)
shop_dm = dict(zip(test_shops, shop_le.transform(test_shops)))


month_le = preprocessing.LabelEncoder()
month_le.fit(range(7,11))
month_dm = dict(zip(range(7,11), month_le.transform(range(7,11))))

In [10]:
values = train.values.tolist()
keys = train.index.values.tolist()
table = []
for i in range(len(item_dm)):
    table.append([])
    for j in range(len(shop_dm)):
        table[i].append([])
        for k in range(34):
            table[i][j].append([0,10])
print("prepared")

prepared


In [11]:
values = train.values.tolist()
keys = train.index.values.tolist()
for i in range(len(train.index)):
    key = keys[i]
    value = values[i]
    table[item_dm[key[1]]][shop_dm[key[0]]][key[2]] = [value[0],value[1]]


In [12]:
for i in range(len(item_dm)):
    table.append([])
    for j in range(len(shop_dm)):
        price = 0
        count = 0
        k = 0
        while price==0 and k < 34:
            if table[i][j][k][1]<9: 
                price = table[i][j][k][0]
            k = k+1
        k = 0
        while k < 34:
            if table[i][j][k][1]<9: 
                price = table[i][j][k][0]
            else:
                table[i][j][k][1] = 0
                table[i][j][k][0] = price
            k = k+1
print("done")
# table[0][0]

done


In [13]:
MAX_ITEM = len(test_items)
MAX_MONTH = 32 
MAX_SHOP = len(test_shops)

month_count = 32
x_train = np.zeros((MAX_ITEM,MAX_SHOP*MAX_MONTH*2), dtype=np.float32)
for i, item in enumerate(table):
    for j, sales in enumerate(item):  
        for k in range(MAX_MONTH):
            x_train[i][j*MAX_MONTH*2+k*2] = table[i][j][k][0]
            x_train[i][j*MAX_MONTH*2+k*2+1] = table[i][j][k][1]
            
y_train = np.zeros((MAX_ITEM,MAX_SHOP), dtype=np.float32)
for i, item in enumerate(table):
    for j, sales in enumerate(item):  
        y_train[i][j] = table[i][j][MAX_MONTH][1]

In [19]:
MAX_ITEM = len(test_items)
MAX_MONTH = 32 
MAX_SHOP = len(test_shops)

month_count = 32
x_val = np.zeros((MAX_ITEM,MAX_SHOP*MAX_MONTH*2), dtype=np.float32)
for i, item in enumerate(table):
    for j, sales in enumerate(item):  
        for k in range(MAX_MONTH):
            x_val[i][j*MAX_MONTH*2+k*2] = table[i][j][k+1][0]
            x_val[i][j*MAX_MONTH*2+k*2+1] = table[i][j][k+1][1]
            


y_val = np.zeros((MAX_ITEM,MAX_SHOP), dtype=np.float32)
for i, item in enumerate(table):
    for j, sales in enumerate(item):  
        y_val[i][j] = table[i][j][MAX_MONTH+1][1]

In [68]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
# model.add(LSTM(1, input_shape=(maxlen, length)))
# model.add(Dense(1, activation='relu'))

model.add(Dense(64, input_dim=(2688), kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

# model.add(Dense(128, input_dim=(2688)))
# model.add(Dense(64))
# model.add(Dense(64))

# model.add(Dense(256))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
# model.add(Dropout(0.3))

# model.add(Dense(16))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
model.add(Dense(MAX_SHOP))
# model.add(Activation('softmax'))


# from keras.optimizers import RMSprop, Adam, SGD, Nadam
# optimizer = RMSprop(lr=0.005)
# optimizer = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
# optimizer = Nadam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='mean_squared_error', optimizer=optimizer)

# model.fit(x_train, y_train, batch_size=2048, epochs=50, validation_data=(x_val, y_val))
model.fit(x_train, y_train, batch_size=2048, epochs=200, validation_data=(x_val, y_val))
# model.fit(x_train, y_train, batch_size=2048, epochs=10)

Build model...
Train on 5100 samples, validate on 5100 samples
Epoch 1/200
5100/5100 [==============================] - 5s 1ms/step - loss: 1.1261 - val_loss: 0.5141
Epoch 2/200
5100/5100 [==============================] - 0s 23us/step - loss: 0.6471 - val_loss: 0.1389
Epoch 3/200
5100/5100 [==============================] - 0s 23us/step - loss: 0.4023 - val_loss: 0.0700
Epoch 4/200
5100/5100 [==============================] - 0s 23us/step - loss: 0.2308 - val_loss: 0.0585
Epoch 5/200
5100/5100 [==============================] - 0s 23us/step - loss: 0.1495 - val_loss: 0.0541
Epoch 6/200
5100/5100 [==============================] - 0s 22us/step - loss: 0.1362 - val_loss: 0.0431
Epoch 7/200
5100/5100 [==============================] - 0s 23us/step - loss: 0.0972 - val_loss: 0.0298
Epoch 8/200
5100/5100 [==============================] - 0s 24us/step - loss: 0.0706 - val_loss: 0.0258
Epoch 9/200
5100/5100 [==============================] - 0s 22us/step - loss: 0.0623 - val_loss: 0.0262
Ep

5100/5100 [==============================] - 0s 22us/step - loss: 0.0152 - val_loss: 0.0141
Epoch 79/200
5100/5100 [==============================] - 0s 23us/step - loss: 0.0153 - val_loss: 0.0141
Epoch 80/200
5100/5100 [==============================] - 0s 24us/step - loss: 0.0153 - val_loss: 0.0141
Epoch 81/200
5100/5100 [==============================] - 0s 22us/step - loss: 0.0156 - val_loss: 0.0141
Epoch 82/200
5100/5100 [==============================] - 0s 22us/step - loss: 0.0154 - val_loss: 0.0141
Epoch 83/200
5100/5100 [==============================] - 0s 24us/step - loss: 0.0157 - val_loss: 0.0141
Epoch 84/200
5100/5100 [==============================] - 0s 24us/step - loss: 0.0155 - val_loss: 0.0141
Epoch 85/200
5100/5100 [==============================] - 0s 22us/step - loss: 0.0149 - val_loss: 0.0141
Epoch 86/200
5100/5100 [==============================] - 0s 22us/step - loss: 0.0150 - val_loss: 0.0140
Epoch 87/200
5100/5100 [==============================] - 0s 22us/st

5100/5100 [==============================] - 0s 24us/step - loss: 0.0137 - val_loss: 0.0125
Epoch 156/200
5100/5100 [==============================] - 0s 22us/step - loss: 0.0129 - val_loss: 0.0123
Epoch 157/200
5100/5100 [==============================] - 0s 22us/step - loss: 0.0132 - val_loss: 0.0121
Epoch 158/200
5100/5100 [==============================] - 0s 23us/step - loss: 0.0139 - val_loss: 0.0119
Epoch 159/200
5100/5100 [==============================] - 0s 22us/step - loss: 0.0130 - val_loss: 0.0119
Epoch 160/200
5100/5100 [==============================] - 0s 24us/step - loss: 0.0135 - val_loss: 0.0119
Epoch 161/200
5100/5100 [==============================] - 0s 25us/step - loss: 0.0138 - val_loss: 0.0118
Epoch 162/200
5100/5100 [==============================] - 0s 23us/step - loss: 0.0133 - val_loss: 0.0118
Epoch 163/200
5100/5100 [==============================] - 0s 23us/step - loss: 0.0126 - val_loss: 0.0118
Epoch 164/200
5100/5100 [==============================] - 0

In [41]:
optimizer = RMSprop(lr=0.0005)
model.compile(loss='mean_squared_error', optimizer=optimizer)

model.fit(x_train, y_train, batch_size=2048, epochs=20, validation_data=(x_val, y_val))

Train on 5100 samples, validate on 5100 samples
Epoch 1/20
5100/5100 [==============================] - 2s 309us/step - loss: 0.0065 - val_loss: 0.0128
Epoch 2/20
5100/5100 [==============================] - 0s 23us/step - loss: 0.0070 - val_loss: 0.0134
Epoch 3/20
5100/5100 [==============================] - 0s 22us/step - loss: 0.0070 - val_loss: 0.0131
Epoch 4/20
5100/5100 [==============================] - 0s 22us/step - loss: 0.0073 - val_loss: 0.0132
Epoch 5/20
5100/5100 [==============================] - 0s 21us/step - loss: 0.0067 - val_loss: 0.0129
Epoch 6/20
5100/5100 [==============================] - 0s 21us/step - loss: 0.0067 - val_loss: 0.0136
Epoch 7/20
5100/5100 [==============================] - 0s 23us/step - loss: 0.0067 - val_loss: 0.0134
Epoch 8/20
5100/5100 [==============================] - 0s 21us/step - loss: 0.0068 - val_loss: 0.0137
Epoch 9/20
5100/5100 [==============================] - 0s 22us/step - loss: 0.0062 - val_loss: 0.0134
Epoch 10/20
5100/5100 [=

In [42]:
MAX_ITEM = len(test_items)
MAX_MONTH = 32 
MAX_SHOP = len(test_shops)

month_count = 32
x_test = np.zeros((MAX_ITEM,MAX_SHOP*MAX_MONTH*2), dtype=np.float32)
for i, item in enumerate(table):
    for j, sales in enumerate(item):  
        for k in range(MAX_MONTH):
            x_test[i][j*MAX_MONTH*2+k*2] = table[i][j][k+2][0]
            x_test[i][j*MAX_MONTH*2+k*2+1] = table[i][j][k+2][1]
            

In [43]:
predict_test = model.predict(x_test)

In [44]:
predict_test[0][0]

-0.01658982

In [45]:
predict_test = cnt_scaler.inverse_transform(predict_test)

In [46]:
predict_test[0][0]

2.5082781

In [47]:
test = pd.read_csv('all/test.csv')
test = test.set_index(['shop_id', 'item_id'])
test['item_cnt_month'] = 0

test2 = test
test2
for index, sentence in enumerate(test2.index.values.tolist()):
#     print(index, sentence)
    (shop_id, item_id) = (sentence[0], sentence[1])
    test.loc[(shop_id, item_id)]['item_cnt_month'] = predict_test[item_dm[item_id]][shop_dm[shop_id]]
    
    

test = test.reset_index().drop(['shop_id', 'item_id'], axis=1)
test.to_csv('submission2.csv', index=False)


In [68]:
import math
from sklearn.metrics import mean_squared_error

# make predictions
# predict_train = model.predict(x_train)
predict_val = model.predict(x_val)
# invert predictions
# predict_train = cnt_scaler.inverse_transform(predict_train)
# yy_train = cnt_scaler.inverse_transform(y_train)
predict_val = cnt_scaler.inverse_transform(predict_val)
yy_val = cnt_scaler.inverse_transform(y_val)
# calculate root mean squared error
# trainScore = math.sqrt(mean_squared_error(predict_train, yy_train))
# print('Train Score: %.2f RMSE' % (trainScore))
valScore = math.sqrt(mean_squared_error(predict_val, yy_val))
print('Test Score: %.2f RMSE' % (valScore))
#For 1 epoch
# Train Score: 2.31 RMSE
# Test Score: 1.42 RMSE
# 6 epoch
# Train Score: 1.89 RMSE
# Test Score: 1.98 RMSE
# 10 epoch
# Train Score: 1.53 RMSE
# Test Score: 2.23 RMSE
# 10 epoch
# 16 LSTM
# Train Score: 1.93 RMSE
# Test Score: 1.90 RMSE
# 10 epoch
# 8 LSTM
# Train Score: 2.31 RMSE
# Test Score: 1.77 RMSE

# 4 LSTM
# Test Score: 1.52 RMSE

# 1 LSTM
# Test Score: 0.98 RMSE

# 4 Dense, size 16, 10 epoch
# 0.82


# 5 Dense, size 16, 12 epoch
# Test Score: 1.36 RMSE

# Dense 16, 8, 8, 8, 8, 
# epoch 8
# Test Score: 0.58 RMSE


Test Score: 1.43 RMSE


In [38]:
print(len(x_fin))
print(len(y_fin))
# print(len(y_train))
# print(len(y_val))

134126
134126


In [39]:
print('Build model...')
model = Sequential()
# model.add(LSTM(1, input_shape=(maxlen, length)))
# model.add(Dense(1, activation='relu'))


# model.add(Dense(64, input_dim=length))
model.add(Dense(16, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(8, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(8, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(8, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(8, input_dim=(length)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))
# model.add(Dense(16))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
model.add(Dense(1))
# model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.005)
model.compile(loss='mean_squared_error', optimizer=optimizer)

model.fit(x_fin, y_fin, batch_size=2048, epochs=5)

Build model...
Epoch 1/5
134126/134126 [==============================] - 5s 39us/step - loss: 0.2463
Epoch 2/5
134126/134126 [==============================] - 3s 25us/step - loss: 0.1808
Epoch 3/5
134126/134126 [==============================] - 3s 25us/step - loss: 0.1682
Epoch 4/5
134126/134126 [==============================] - 3s 25us/step - loss: 0.1671
Epoch 5/5
134126/134126 [==============================] - 3s 25us/step - loss: 0.1524


In [38]:
model.save("my_model.h5") 

In [ ]:
model = keras.models.load_model("my_model.h5") 

In [95]:
del predict_train
del predict_val

In [58]:
model.fit(x_val, y_val, batch_size=2048, epochs=13)

Epoch 1/13
111404/111404 [==============================] - 19s 172us/step - loss: 9.2909
Epoch 2/13
111404/111404 [==============================] - 19s 172us/step - loss: 7.6454
Epoch 3/13
111404/111404 [==============================] - 19s 173us/step - loss: 6.8270
Epoch 4/13
111404/111404 [==============================] - 19s 173us/step - loss: 6.0881
Epoch 5/13
111404/111404 [==============================] - 19s 174us/step - loss: 5.6661
Epoch 6/13
111404/111404 [==============================] - 19s 173us/step - loss: 5.3044
Epoch 7/13
111404/111404 [==============================] - 19s 173us/step - loss: 5.0030
Epoch 8/13
111404/111404 [==============================] - 19s 172us/step - loss: 4.6944
Epoch 9/13
111404/111404 [==============================] - 19s 173us/step - loss: 4.3458
Epoch 10/13
111404/111404 [==============================] - 19s 171us/step - loss: 4.1903
Epoch 11/13
111404/111404 [==============================] - 19s 172us/step - loss: 3.9588
Epoch 12

In [40]:
predict_test = model.predict(x_test)
predict_test = cnt_scaler.inverse_transform(predict_test)

In [41]:

test = pd.read_csv('all/test.csv')
test = test.set_index(['shop_id', 'item_id'])
test['item_cnt_month'] = 0

for index, sentence in enumerate(x_test_o):
    (shop_id, item_id) = (sentence[0]['shop_id'], sentence[0]['item_id'])
    test.loc[(shop_id, item_id)]['item_cnt_month'] = predict_test[index]
    
    

test = test.reset_index().drop(['shop_id', 'item_id'], axis=1)
test.to_csv('submission2.csv', index=False)
